In [3]:
from pyspark.sql import SparkSession

def create_spark(app_name="5400-news-elt"):
    spark = (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")
        .config("spark.driver.memory", "12g")
        .config("spark.executor.memory", "12g")
        .config("spark.sql.shuffle.partitions", "400")
        .config("spark.default.parallelism", "400")
        .config("spark.sql.files.maxRecordsPerFile", "200000")
        .config("spark.sql.parquet.compression.codec", "snappy")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("WARN")
    print("🔥 Spark Ready:", spark.version)
    return spark

In [4]:
import os
from pyspark.sql import functions as F
from pyspark.sql import types as T

BASE = r"D:\Columbia\Fall2025\5400\project\layer"
CSV_PATH = r"D:\Columbia\Fall2025\5400\project\All_external.csv"

BRONZE = os.path.join(BASE, "bronze")
SILVER = os.path.join(BASE, "silver")
GOLD = os.path.join(BASE, "gold")
NLP_SILVER = os.path.join(BASE, "silver_for_nlp")


In [5]:
def build_bronze(spark):
    os.makedirs(BRONZE, exist_ok=True)

    print("📥 Loading CSV...")
    df = (
        spark.read.format("csv")
        .options(
            header="true",
            inferSchema="true",
            multiLine="true",
            escape="\"",
            quote="\"",
            mode="PERMISSIVE"
        )
        .load(CSV_PATH)
    )

    # Cleaning string columns
    for c, t in df.dtypes:
        if t == "string":
            df = df.withColumn(c, F.trim(F.col(c)))

    # date
    if "Date" in df.columns:
        df = df.withColumn("Date", F.to_date("Date", "yyyy-MM-dd"))

    df = df.repartition(400)

    print("💾 Writing Bronze...")
    df.write.mode("overwrite").parquet(BRONZE)
    print("🥉 Bronze Ready:", BRONZE)

    return df

In [ ]:
def build_silver(spark):
    os.makedirs(SILVER, exist_ok=True)
    df = spark.read.parquet(BRONZE)

    # key fields not null
    if "Date" in df.columns:
        df = df.filter(F.col("Date").isNotNull())
    if "Stock_symbol" in df.columns:
        df = df.filter(F.col("Stock_symbol").isNotNull())

    # drop duplicates
    df = df.dropDuplicates()

    # standardize text fields
    if "Publisher" in df.columns:
        df = df.withColumn("Publisher_norm", F.upper("Publisher"))
    if "Author" in df.columns:
        df = df.withColumn("Author_norm", F.upper("Author"))

    # unique ID
    df = df.withColumn("news_id", F.monotonically_increasing_id())

    # repartition
    df = df.repartition(400, "Stock_symbol", "Date")

    print("💾 Writing Silver...")
    df.write.mode("overwrite").parquet(SILVER)
    print("🥈 Silver Ready:", SILVER)

    return df

In [ ]:
def build_gold(spark):
    os.makedirs(GOLD, exist_ok=True)
    df = spark.read.parquet(SILVER)

    df = df.withColumn("title_len", F.length("Article_title"))

    agg_cols = [
        F.count("*").alias("article_count"),
        F.countDistinct("Publisher").alias("publisher_count"),
        F.avg("title_len").alias("avg_title_len"),
    ]

    for col in ["Textrank_summary", "Lsa_summary", "Luhn_summary", "Lexrank_summary"]:
        if col in df.columns:
            agg_cols.append(F.first(col, ignorenulls=True).alias("sample_summary"))
            break

    gold_df = df.groupBy("Date", "Stock_symbol").agg(*agg_cols)
    gold_df = gold_df.repartition(200, "Stock_symbol")

    print("💾 Writing Gold...")
    gold_df.write.mode("overwrite").parquet(GOLD)
    print("🥇 Gold Ready:", GOLD)

    return gold_df

In [6]:
def build_silver_nlp(spark):
    os.makedirs(NLP_SILVER, exist_ok=True)

    df = spark.read.parquet(SILVER)

    # auto select text column
    df = df.withColumn(
        "text",
        F.coalesce(
            F.col("Article"),
            F.col("Textrank_summary"),
            F.col("Lsa_summary"),
            F.col("Luhn_summary"),
            F.col("Lexrank_summary"),
            F.col("Article_title"),   # solve empty text cases
        )
    )

    # filter non-empty text
    df = df.filter(F.col("text").isNotNull() & (F.length(F.trim("text")) > 0))

    df = df.select(
        "news_id", "Date", "Article_title", "Stock_symbol",
        "Publisher", "Author", "Url", "text"
    ).repartition(200)

    df.write.mode("overwrite").parquet(NLP_SILVER)

    print("🧠 Silver_for_nlp done:", NLP_SILVER)
    return df

In [7]:
spark = create_spark()

bronze_df = build_bronze(spark)
silver_df = build_silver(spark)
gold_df   = build_gold(spark)
nlp_df    = build_silver_nlp(spark)

🔥 Spark Ready: 3.3.1
📥 Loading CSV...
💾 Writing Bronze...
🥉 Bronze Ready: D:\Columbia\Fall2025\5400\project\layer\bronze
💾 Writing Silver...
🥈 Silver Ready: D:\Columbia\Fall2025\5400\project\layer\silver
💾 Writing Gold...
🥇 Gold Ready: D:\Columbia\Fall2025\5400\project\layer\gold
🧠 Silver_for_nlp done: D:\Columbia\Fall2025\5400\project\layer\silver_for_nlp


In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

SILVER = r"D:\Columbia\Fall2025\5400\project\layer\silver"

df = spark.read.parquet(SILVER)

print(df.columns)

df.printSchema()

['Date', 'Article_title', 'Stock_symbol', 'Url', 'Publisher', 'Author', 'Article', 'Lsa_summary', 'Luhn_summary', 'Textrank_summary', 'Lexrank_summary', 'Publisher_norm', 'Author_norm', 'news_id']
root
 |-- Date: date (nullable = true)
 |-- Article_title: string (nullable = true)
 |-- Stock_symbol: string (nullable = true)
 |-- Url: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- Article: string (nullable = true)
 |-- Lsa_summary: string (nullable = true)
 |-- Luhn_summary: string (nullable = true)
 |-- Textrank_summary: string (nullable = true)
 |-- Lexrank_summary: string (nullable = true)
 |-- Publisher_norm: string (nullable = true)
 |-- Author_norm: string (nullable = true)
 |-- news_id: long (nullable = true)



In [9]:
import pyarrow.dataset as ds

NLP_SILVER = r"D:\Columbia\Fall2025\5400\project\layer\silver_for_nlp"
d = ds.dataset(NLP_SILVER, format="parquet")

print("NLP rows:", d.count_rows())
print("Schema:", d.schema)

NLP rows: 3242351
Schema: news_id: int64
Date: date32[day]
Article_title: string
Stock_symbol: string
Publisher: string
Author: string
Url: string
text: string
-- schema metadata --
org.apache.spark.version: '3.3.1'
org.apache.spark.sql.parquet.row.metadata: '{"type":"struct","fields":[{"' + 517


#### NLP(Silver)


In [10]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

SILVER = r"D:\Columbia\Fall2025\5400\project\layer\silver"
BRONZE = r"D:\Columbia\Fall2025\5400\project\layer\bronze"

silver_df = spark.read.parquet(SILVER)
bronze_df = spark.read.parquet(BRONZE)

print("Bronze rows:", bronze_df.count())
print("Silver rows:", silver_df.count())
silver_df.select("Date").show(5, False)

Bronze rows: 13057514
Silver rows: 3242351
+----------+
|Date      |
+----------+
|2014-04-23|
|2012-02-15|
|2017-04-03|
|2017-02-15|
|2018-03-13|
+----------+
only showing top 5 rows



In [1]:
# FinBERT sentiment
import os
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

# Paths
BASE = r"D:\Columbia\Fall2025\5400\project\layer"
NLP_SILVER = os.path.join(BASE, "silver_for_nlp")
SENTIMENT_SAMPLE = os.path.join(BASE, "sentiment_sample")

# Local model cache directory
MODEL_CACHE = r"D:\Columbia\Fall2025\5400\project\models"
os.makedirs(MODEL_CACHE, exist_ok=True)
os.makedirs(SENTIMENT_SAMPLE, exist_ok=True)

# GPU detection
if torch.cuda.is_available() and torch.cuda.device_count() > 0:
    device = 0
    print("✅ Using GPU device 0 for FinBERT")
else:
    device = -1
    print("⚠ CUDA not available, running FinBERT on CPU")

# FinBERT tokenizer & model (with cache_dir)
tokenizer = AutoTokenizer.from_pretrained(
    "ProsusAI/finbert",
    cache_dir=MODEL_CACHE,
)

model = AutoModelForSequenceClassification.from_pretrained(
    "ProsusAI/finbert",
    cache_dir=MODEL_CACHE,
)

sentiment_pipe = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=device,
)


# Load dataset from parquet
dataset = ds.dataset(NLP_SILVER, format="parquet")
total_rows = dataset.count_rows()
print("🧠 Total rows for NLP:", total_rows)

BATCH_ROWS = 4000         # number of rows read from parquet each time
PIPELINE_BATCH = 32       # batch_size for HF pipeline
MAX_LENGTH = 256
MAX_ROWS = 3_242_351
processed_rows = 0

batches = dataset.to_batches(batch_size=BATCH_ROWS)

# Main loop
for i, batch in enumerate(batches):

    if processed_rows >= MAX_ROWS:
        print(f"⚡ Reached MAX_ROWS={MAX_ROWS}, stopping early.")
        break

    table = pa.Table.from_batches([batch])
    pdf = table.to_pandas()

    # Filter invalid text
    pdf = pdf[pdf["text"].notna() & (pdf["text"].astype(str).str.strip() != "")]
    if pdf.empty:
        continue

    texts = pdf["text"].astype(str).tolist()

    # 🔥 Run FinBERT sentiment
    preds = sentiment_pipe(
        texts,
        batch_size=PIPELINE_BATCH,
        truncation=True,
        max_length=MAX_LENGTH,
    )

    # FinBERT labels: positive / neutral / negative
    pdf["sentiment_label"] = [p["label"] for p in preds]
    pdf["sentiment_score"] = [float(p["score"]) for p in preds]

    # Signed sentiment score:
    #   positive → +score
    #   negative → -score
    #   neutral  → 0
    pdf["sentiment_score_signed"] = pdf.apply(
        lambda row: (
            row["sentiment_score"]
            if row["sentiment_label"] == "positive"
            else (-row["sentiment_score"] if row["sentiment_label"] == "negative" else 0.0)
        ),
        axis=1,
    )

    # Output columns
    out_pdf = pdf[["news_id", "sentiment_label",
                   "sentiment_score", "sentiment_score_signed"]]

    # Write to parquet
    out_table = pa.Table.from_pandas(out_pdf, preserve_index=False)
    out_file = os.path.join(SENTIMENT_SAMPLE, f"part-{i:05d}.parquet")
    pq.write_table(out_table, out_file)

    processed_rows += len(out_pdf)
    print(f"✅ Batch {i}: wrote {len(out_pdf)} rows, total processed {processed_rows}")

print("🎉 FinBERT Sentiment SAMPLE Completed:", SENTIMENT_SAMPLE)
print("Total processed rows:", processed_rows)


✅ Using GPU device 0 for FinBERT


Device set to use cuda:0


🧠 Total rows for NLP: 3242351
✅ Batch 0: wrote 4000 rows, total processed 4000
✅ Batch 1: wrote 4000 rows, total processed 8000
✅ Batch 2: wrote 4000 rows, total processed 12000
✅ Batch 3: wrote 4000 rows, total processed 16000
✅ Batch 4: wrote 226 rows, total processed 16226
✅ Batch 5: wrote 4000 rows, total processed 20226
✅ Batch 6: wrote 4000 rows, total processed 24226
✅ Batch 7: wrote 4000 rows, total processed 28226
✅ Batch 8: wrote 4000 rows, total processed 32226


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ Batch 9: wrote 224 rows, total processed 32450
✅ Batch 10: wrote 4000 rows, total processed 36450
✅ Batch 11: wrote 4000 rows, total processed 40450
✅ Batch 12: wrote 4000 rows, total processed 44450
✅ Batch 13: wrote 4000 rows, total processed 48450
✅ Batch 14: wrote 223 rows, total processed 48673
✅ Batch 15: wrote 4000 rows, total processed 52673
✅ Batch 16: wrote 4000 rows, total processed 56673
✅ Batch 17: wrote 4000 rows, total processed 60673
✅ Batch 18: wrote 4000 rows, total processed 64673
✅ Batch 19: wrote 222 rows, total processed 64895
✅ Batch 20: wrote 4000 rows, total processed 68895
✅ Batch 21: wrote 4000 rows, total processed 72895
✅ Batch 22: wrote 4000 rows, total processed 76895
✅ Batch 23: wrote 4000 rows, total processed 80895
✅ Batch 24: wrote 227 rows, total processed 81122
✅ Batch 25: wrote 4000 rows, total processed 85122
✅ Batch 26: wrote 4000 rows, total processed 89122
✅ Batch 27: wrote 4000 rows, total processed 93122
✅ Batch 28: wrote 4000 rows, total p

In [1]:
# # distilbert(sub for FinBERT)
# import os
# import pyarrow.dataset as ds
# import pyarrow.parquet as pq
# import pyarrow as pa
# import pandas as pd
# from transformers import pipeline
# import torch
#
# BASE = r"D:\Columbia\Fall2025\5400\project\layer"
# NLP_SILVER = os.path.join(BASE, "silver_for_nlp")
# SENTIMENT_SAMPLE = os.path.join(BASE, "sentiment_sample")
# os.makedirs(SENTIMENT_SAMPLE, exist_ok=True)
#
# # 1. GPU
# if torch.cuda.is_available() and torch.cuda.device_count() > 0:
#     device = 0
#     print("✅ Using GPU device 0 for sentiment.")
# else:
#     device = -1
#     print("⚠ 没有可用 CUDA 设备，使用 CPU。")
#
# # 2. HF pipeline
# sentiment_pipe = pipeline(
#     "sentiment-analysis",
#     model="distilbert-base-uncased-finetuned-sst-2-english",
#     device=device,
# )
#
# # 3. prepare dataset
# dataset = ds.dataset(NLP_SILVER, format="parquet")
# total_rows = dataset.count_rows()
# print("🧠 NLP total rows:", total_rows)
#
# BATCH_ROWS = 4000        # read 4000 rows at a time from parquet
# PIPELINE_BATCH = 32      # batch_size
# MAX_LENGTH = 256
# MAX_ROWS = 3242351
# processed_rows = 0
#
# batches = dataset.to_batches(batch_size=BATCH_ROWS)
#
# for i, batch in enumerate(batches):
#     if processed_rows >= MAX_ROWS:
#         print(f"⚡ 已达到 MAX_ROWS={MAX_ROWS}，提前结束。")
#         break
#
#     table = pa.Table.from_batches([batch])
#     pdf = table.to_pandas()
#
#     pdf = pdf[pdf["text"].notna() & (pdf["text"].astype(str).str.strip() != "")]
#     if pdf.empty:
#         continue
#
#     texts = pdf["text"].astype(str).tolist()
#
#     preds = sentiment_pipe(
#         texts,
#         batch_size=PIPELINE_BATCH,
#         truncation=True,
#         max_length=MAX_LENGTH,
#     )
#
#     pdf["sentiment_label"] = [p["label"] for p in preds]
#     pdf["sentiment_score"] = [float(p["score"]) for p in preds]
#     pdf["sentiment_score_signed"] = pdf["sentiment_score"].where(
#         pdf["sentiment_label"] == "POSITIVE",
#         -pdf["sentiment_score"],
#     )
#
#     out_pdf = pdf[["news_id", "sentiment_label",
#                    "sentiment_score", "sentiment_score_signed"]]
#     out_table = pa.Table.from_pandas(out_pdf, preserve_index=False)
#
#     out_file = os.path.join(SENTIMENT_SAMPLE, f"part-{i:05d}.parquet")
#     pq.write_table(out_table, out_file)
#
#     processed_rows += len(out_pdf)
#     print(f"✅ batch {i}, now written {len(out_pdf)} rows，counted {processed_rows} rows")
#
# print("🎉 Sentiment SAMPLE Done:", SENTIMENT_SAMPLE)
# print("实际处理行数:", processed_rows)


✅ Using GPU device 0 for sentiment.


Device set to use cuda:0


🧠 NLP total rows: 3242351
✅ batch 0, 当前写出 4000 行，累计 4000 行
✅ batch 1, 当前写出 4000 行，累计 8000 行
✅ batch 2, 当前写出 4000 行，累计 12000 行
✅ batch 3, 当前写出 4000 行，累计 16000 行
✅ batch 4, 当前写出 226 行，累计 16226 行
✅ batch 5, 当前写出 4000 行，累计 20226 行
✅ batch 6, 当前写出 4000 行，累计 24226 行
✅ batch 7, 当前写出 4000 行，累计 28226 行


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ batch 8, 当前写出 4000 行，累计 32226 行
✅ batch 9, 当前写出 224 行，累计 32450 行
✅ batch 10, 当前写出 4000 行，累计 36450 行
✅ batch 11, 当前写出 4000 行，累计 40450 行
✅ batch 12, 当前写出 4000 行，累计 44450 行
✅ batch 13, 当前写出 4000 行，累计 48450 行
✅ batch 14, 当前写出 223 行，累计 48673 行
✅ batch 15, 当前写出 4000 行，累计 52673 行
✅ batch 16, 当前写出 4000 行，累计 56673 行
✅ batch 17, 当前写出 4000 行，累计 60673 行
✅ batch 18, 当前写出 4000 行，累计 64673 行
✅ batch 19, 当前写出 222 行，累计 64895 行
✅ batch 20, 当前写出 4000 行，累计 68895 行
✅ batch 21, 当前写出 4000 行，累计 72895 行
✅ batch 22, 当前写出 4000 行，累计 76895 行
✅ batch 23, 当前写出 4000 行，累计 80895 行
✅ batch 24, 当前写出 227 行，累计 81122 行
✅ batch 25, 当前写出 4000 行，累计 85122 行
✅ batch 26, 当前写出 4000 行，累计 89122 行
✅ batch 27, 当前写出 4000 行，累计 93122 行
✅ batch 28, 当前写出 4000 行，累计 97122 行
✅ batch 29, 当前写出 222 行，累计 97344 行
✅ batch 30, 当前写出 4000 行，累计 101344 行
✅ batch 31, 当前写出 4000 行，累计 105344 行
✅ batch 32, 当前写出 4000 行，累计 109344 行
✅ batch 33, 当前写出 4000 行，累计 113344 行
✅ batch 34, 当前写出 224 行，累计 113568 行
✅ batch 35, 当前写出 4000 行，累计 117568 行
✅ batch 36, 当前写出 4000 

In [1]:
import os, pyarrow.parquet as pq

SENTIMENT_SAMPLE = r"D:\Columbia\Fall2025\5400\project\layer\sentiment_sample"
files = sorted(os.listdir(SENTIMENT_SAMPLE))
print("Number of files:", len(files))
print("Example:", files[:5])

t = pq.read_table(os.path.join(SENTIMENT_SAMPLE, files[0]))
print("Samples：")
print(t.to_pandas().head())


Number of files: 1000
Example: ['part-00000.parquet', 'part-00001.parquet', 'part-00002.parquet', 'part-00003.parquet', 'part-00004.parquet']
Samples：
         news_id sentiment_label  sentiment_score  sentiment_score_signed
0  2284922605296         neutral         0.941989                0.000000
1  1967095022423         neutral         0.821760                0.000000
2  2422361556929         neutral         0.643113                0.000000
3  2181843394301        positive         0.688343                0.688343
4  1451698949482         neutral         0.893164                0.000000


In [2]:
# aggregation
import os
from pyspark.sql import SparkSession, functions as F

def create_spark(app_name="5400-news-elt-join-sample"):
    spark = (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")
        .config("spark.driver.memory", "12g")
        .config("spark.executor.memory", "12g")
        .config("spark.sql.shuffle.partitions", "400")
        .config("spark.default.parallelism", "400")
        .config("spark.sql.parquet.compression.codec", "snappy")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("WARN")
    return spark

BASE = r"D:\Columbia\Fall2025\5400\project\layer"
SILVER = os.path.join(BASE, "silver")
SENTIMENT_SAMPLE = os.path.join(BASE, "sentiment_sample")
SILVER_WITH_SENT_SAMPLE = os.path.join(BASE, "silver_with_sentiment_sample")

spark = create_spark()

silver_df = spark.read.parquet(SILVER)
sent_df = spark.read.parquet(SENTIMENT_SAMPLE)

print("Silver layer row number：", silver_df.count())
print("Sentiment row number：", sent_df.count())

# left join
silver_with_sent = silver_df.join(sent_df, on="news_id", how="left")

silver_with_sent = silver_with_sent.repartition(200, "Stock_symbol", "Date")

(
    silver_with_sent
    .write
    .mode("overwrite")
    .parquet(SILVER_WITH_SENT_SAMPLE)
)

print("✅ silver_with_sentiment_sample：", SILVER_WITH_SENT_SAMPLE)

test_df = spark.read.parquet(SILVER_WITH_SENT_SAMPLE)
test_df.select("news_id","Article_title","sentiment_label",
               "sentiment_score_signed").show(20, False)

spark.stop()

Silver layer row number： 3242351
Sentiment row number： 3242351
✅ silver_with_sentiment_sample： D:\Columbia\Fall2025\5400\project\layer\silver_with_sentiment_sample
+-------------+----------------------------------------------------------------------------------------------------------------------------------+---------------+----------------------+
|news_id      |Article_title                                                                                                                     |sentiment_label|sentiment_score_signed|
+-------------+----------------------------------------------------------------------------------------------------------------------------------+---------------+----------------------+
|317827582759 |Benzinga's Top Upgrades, Downgrades For July 1, 2019                                                                              |neutral        |0.0                   |
|386547061560 |Are Hedge Funds Setup For A Short Squeeze?                                   

### Final Gold NLP


In [8]:
import os
from pyspark.sql import SparkSession, functions as F


def create_spark(app_name="5400-news-gold-sent-sample"):
    spark = (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")
        .config("spark.driver.memory", "12g")
        .config("spark.executor.memory", "12g")
        .config("spark.sql.shuffle.partitions", "400")
        .config("spark.default.parallelism", "400")
        .config("spark.sql.parquet.compression.codec", "snappy")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("WARN")
    return spark


# Paths

BASE = r"D:\Columbia\Fall2025\5400\project\layer"

GOLD = os.path.join(BASE, "gold")   # aggregated by (Date, Stock_symbol)
SILVER_WITH_SENT_SAMPLE = os.path.join(BASE, "silver_with_sentiment_sample")
GOLD_WITH_SENT_SAMPLE = os.path.join(BASE, "gold_with_sentiment_sample")

spark = create_spark()


# 1) Read data
gold_df = spark.read.parquet(GOLD)
silver_sent_raw = spark.read.parquet(SILVER_WITH_SENT_SAMPLE)

print("Gold rows (aggregated):", gold_df.count())
print("Silver + Sentiment sample rows (article level):", silver_sent_raw.count())
print("Gold columns:", gold_df.columns)
print("Silver_with_sentiment_sample columns:", silver_sent_raw.columns)

# ---------------------------------------------------
# news_id, Date, Stock_symbol, Article_title, Publisher, Url,
# sentiment_label, sentiment_score_signed
# ---------------------------------------------------
silver_sent = silver_sent_raw.select(
    "news_id",
    "Date",
    "Stock_symbol",
    "Article_title",
    "Publisher",
    "Url",
    "sentiment_label",
    "sentiment_score_signed",
)

# 2) Aggregate sentiment by (Date, Stock_symbol)
sent_agg = (
    silver_sent
    .groupBy("Date", "Stock_symbol")
    .agg(
        # average signed sentiment per (Date, Stock_symbol)
        F.avg("sentiment_score_signed").alias("avg_sentiment_score"),

        # count positive / negative news
        F.sum(
            F.when(F.col("sentiment_label") == "POSITIVE", 1).otherwise(0)
        ).alias("pos_article_count"),
        F.sum(
            F.when(F.col("sentiment_label") == "NEGATIVE", 1).otherwise(0)
        ).alias("neg_article_count"),

        # total news with sentiment
        F.count(F.col("sentiment_label")).alias("sentiment_article_count"),
    )
)

# add positive / negative ratios
sent_agg = (
    sent_agg
    .withColumn(
        "pos_ratio",
        F.when(
            F.col("sentiment_article_count") > 0,
            F.col("pos_article_count") / F.col("sentiment_article_count")
        ).otherwise(F.lit(None))
    )
    .withColumn(
        "neg_ratio",
        F.when(
            F.col("sentiment_article_count") > 0,
            F.col("neg_article_count") / F.col("sentiment_article_count")
        ).otherwise(F.lit(None))
    )
)

print("Sentiment aggregation finished. Columns:", sent_agg.columns)

# 3) Join sentiment aggregation back to article-level table
silver_with_sent_agg = (
    silver_sent.alias("ss")
    .join(
        sent_agg.alias("sa"),
        on=["Date", "Stock_symbol"],
        how="left"
    )
)

print("After joining sent_agg, columns:", silver_with_sent_agg.columns)

# 4) Join with GOLD aggregated metrics
#    GOLD has: article_count, publisher_count, avg_title_len, etc.
gold_with_sent = (
    silver_with_sent_agg.alias("s")
    .join(
        gold_df.alias("g"),
        on=["Date", "Stock_symbol"],
        how="left"
    )
)

# 5) Sentiment category based on avg_sentiment_score
gold_with_sent = gold_with_sent.withColumn(
    "sentiment_category",
    F.when(F.col("avg_sentiment_score") >= 0.2, "POSITIVE")
     .when(F.col("avg_sentiment_score") <= -0.2, "NEGATIVE")
     .otherwise("NEUTRAL")
)

# 6) Author / summaries / *_norm / sample_summary
gold_with_sent = gold_with_sent.select(
    "news_id",
    "Date",
    "Stock_symbol",
    "Article_title",
    "Publisher",
    "Url",
    "sentiment_label",
    "sentiment_score_signed",

    # aggregated sentiment metrics
    "avg_sentiment_score",
    "pos_article_count",
    "neg_article_count",
    "sentiment_article_count",
    "pos_ratio",
    "neg_ratio",

    # GOLD metrics
    "article_count",
    "publisher_count",
    "avg_title_len",

    # category
    "sentiment_category",
)

print("Final pruned columns:", gold_with_sent.columns)


# 7) Repartition & write out
gold_with_sent = gold_with_sent.repartition(100, "Stock_symbol")

(
    gold_with_sent
    .write
    .mode("overwrite")
    .parquet(GOLD_WITH_SENT_SAMPLE)
)

print("🏆 gold_with_sentiment_sample written to:", GOLD_WITH_SENT_SAMPLE)

Gold rows (aggregated): 1896033
Silver + Sentiment sample rows (article level): 3242351
Gold columns: ['Date', 'Stock_symbol', 'article_count', 'publisher_count', 'avg_title_len', 'sample_summary']
Silver_with_sentiment_sample columns: ['news_id', 'Date', 'Article_title', 'Stock_symbol', 'Url', 'Publisher', 'Author', 'Article', 'Lsa_summary', 'Luhn_summary', 'Textrank_summary', 'Lexrank_summary', 'Publisher_norm', 'Author_norm', 'sentiment_label', 'sentiment_score', 'sentiment_score_signed']
Sentiment aggregation finished. Columns: ['Date', 'Stock_symbol', 'avg_sentiment_score', 'pos_article_count', 'neg_article_count', 'sentiment_article_count', 'pos_ratio', 'neg_ratio']
After joining sent_agg, columns: ['Date', 'Stock_symbol', 'news_id', 'Article_title', 'Publisher', 'Url', 'sentiment_label', 'sentiment_score_signed', 'avg_sentiment_score', 'pos_article_count', 'neg_article_count', 'sentiment_article_count', 'pos_ratio', 'neg_ratio']
Final pruned columns: ['news_id', 'Date', 'Stock_s

In [9]:
import os
import pandas as pd
import pyarrow.parquet as pq
import pyarrow.dataset as ds

SILVER_WITH_SENT_SAMPLE = r"D:\Columbia\Fall2025\5400\project\layer\silver_with_sentiment_sample"
OUTPUT_CSV = r"D:\Columbia\Fall2025\5400\project\silver_with_sentiment_sample.csv"

dataset = ds.dataset(SILVER_WITH_SENT_SAMPLE, format="parquet")
table = dataset.to_table()
df = table.to_pandas()

df.to_csv(OUTPUT_CSV, index=False)
print("CSV out：", OUTPUT_CSV)

CSV out： D:\Columbia\Fall2025\5400\project\silver_with_sentiment_sample.csv


In [10]:
import os
import pandas as pd
import pyarrow.parquet as pq
import pyarrow.dataset as ds

GOLD_WITH_SENT_SAMPLE = r"D:\Columbia\Fall2025\5400\project\layer\gold_with_sentiment_sample"
OUTPUT_CSV = r"D:\Columbia\Fall2025\5400\project\gold_with_sentiment_sample.csv"

dataset = ds.dataset(GOLD_WITH_SENT_SAMPLE, format="parquet")
table = dataset.to_table()
df = table.to_pandas()

df.to_csv(OUTPUT_CSV, index=False)
print("CSV out：", OUTPUT_CSV)

CSV out： D:\Columbia\Fall2025\5400\project\gold_with_sentiment_sample.csv
